In [2]:

import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

from keras.layers import SimpleRNN
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
import keras
from keras.models import Sequential, load_model
from keras.layers.core import Dense
from keras.optimizers import RMSprop, Adam, SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

In [68]:
!pip install tensorflow==1.15

ERROR: Could not find a version that satisfies the requirement tensorflow==1.15 (from versions: 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.2.1, 2.2.2, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0, 2.3.1, 2.3.2, 2.4.0rc0, 2.4.0rc1, 2.4.0rc2, 2.4.0rc3, 2.4.0rc4, 2.4.0, 2.4.1)
ERROR: No matching distribution found for tensorflow==1.15


In [2]:
df = pd.read_csv("data/AIDA_CRM_20210312.csv", encoding = "UTF8", thousands = ",")
df

,Unnamed: 0,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,옵션입력,앰플
0,0,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
1,1,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
2,2,sjh3460,3.0,0,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
3,3,kaeng2da,3.0,0,복합성,좁쌀여드름,수분앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
4,4,gmlwlss625,2.0,0,복합성,"과잉피지,여드름,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
...,...,...,...,...,...,...,...,...,...,...
34651,34651,nadaseonho,3.0,0,복합성,"건조함,홍조&예민",진정앰플,[시앙제이X아이다] 인생 버블토너 1차 공구!,필수 옵션 프로폴리스 진정토너 옵션 머그워트 클리어토너 옵션 ...,[' 진정앰플울트라']
34652,34652,rarrarra12,3.0,0,지성,"과잉피지,여드름,좁쌀여드름",진정앰플,BEST앰플3종(수분/미백/진정),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,"[' 수분앰플', ' 미백앰플', ' 진정앰플']"
34653,34653,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분앰플라이트', ' 진정앰플', ' 부스터앰플', '..."
34654,34654,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분앰플라이트', ' 진정앰플', ' 부스터앰플', '..."


In [3]:
df["피부타입"].unique()

array(['건성', '지성', '복합성', '모름'], dtype=object)

In [4]:
encoding = pd.get_dummies(df['피부타입'])

In [5]:
df = pd.concat([encoding, df], axis=1)

In [6]:
del df['Unnamed: 0'], df['피부타입']

In [11]:
df

,건성,모름,복합성,지성,아이디,나이,성별,피부고민,선호제품,주문상품명,옵션입력,앰플
0,1,0,0,0,thsk486,4.0,0,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
1,1,0,0,0,thsk486,4.0,0,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
2,0,0,0,1,sjh3460,3.0,0,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
3,0,0,1,0,kaeng2da,3.0,0,좁쌀여드름,수분앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
4,0,0,1,0,gmlwlss625,2.0,0,"과잉피지,여드름,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진..."
...,...,...,...,...,...,...,...,...,...,...,...,...
34651,0,0,1,0,nadaseonho,3.0,0,"건조함,홍조&예민",진정앰플,[시앙제이X아이다] 인생 버블토너 1차 공구!,필수 옵션 프로폴리스 진정토너 옵션 머그워트 클리어토너 옵션 ...,[' 진정앰플울트라']
34652,0,0,0,1,rarrarra12,3.0,0,"과잉피지,여드름,좁쌀여드름",진정앰플,BEST앰플3종(수분/미백/진정),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,"[' 수분앰플', ' 미백앰플', ' 진정앰플']"
34653,1,0,0,0,gy6686,1.0,0,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분앰플라이트', ' 진정앰플', ' 부스터앰플', '..."
34654,1,0,0,0,gy6686,1.0,0,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분앰플라이트', ' 진정앰플', ' 부스터앰플', '..."


In [40]:
faceworry = ['건조함','과잉피지','여드름','좁쌀','홍조&예민','기미&잡티']

In [46]:
len(df)
df['건조함'] = None
df['과잉피지'] = None
df['여드름'] = None
df['좁쌀'] = None
df['홍조&예민'] = None
df['기미&잡티'] = None

In [52]:
del df['좁쌀여드름']

In [55]:
del df['피부고민_replace']

In [57]:
del df['worry_replace']

In [51]:
# 피부고민 숫자형태로 인코딩
for i in range(len(df)):
    for j in faceworry:
        if j in df['피부고민'][i]:
            df[j][i] = 1
        else:
            df[j][i] = 0
            

        

<ipython-input-51-3c70c911a0f8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j][i] = 1
<ipython-input-51-3c70c911a0f8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j][i] = 0


In [50]:
# 좁쌀여드름 -> 좁쌀
df['피부고민'] = df['피부고민'].str.replace(pat='좁쌀여드름', repl='좁쌀', regex=False)

In [60]:
df = df[['아이디', '나이', '성별', '피부고민', '선호제품', '주문상품명', '옵션입력', '앰플', '건성', '지성', '복합성',
    '모름', '건조함', '과잉피지', '여드름', '좁쌀', '홍조&예민', '기미&잡티']]

In [62]:
del df['피부고민']

In [66]:
df.to_csv('data/CRM_mid.csv', encoding='utf-8-sig')

In [27]:
df

,아이디,나이,성별,선호제품,주문상품명,옵션입력,앰플,건성,지성,복합성,모름,건조함,과잉피지,여드름,좁쌀,홍조&예민,기미&잡티
0,thsk486,4.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",1,0,0,0,1,0,0,1,1,0
1,thsk486,4.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",1,0,0,0,1,0,0,1,1,0
2,sjh3460,3.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,1,0,0,0,0,1,0,0,0
3,kaeng2da,3.0,0,수분앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,0,1,0,0,0,0,1,0,0
4,gmlwlss625,2.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,0,1,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34651,nadaseonho,3.0,0,진정앰플,[시앙제이X아이다] 인생 버블토너 1차 공구!,필수 옵션 프로폴리스 진정토너 옵션 머그워트 클리어토너 옵션 ...,[' 진정앰플울트라'],0,0,1,0,1,0,0,0,1,0
34652,rarrarra12,3.0,0,진정앰플,BEST앰플3종(수분/미백/진정),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,"[' 수분앰플', ' 미백앰플', ' 진정앰플']",0,1,0,0,0,1,1,1,0,0
34653,gy6686,1.0,0,진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분라이트', ' 진정앰플', ' 부스터앰플', ' 미...",1,0,0,0,1,0,0,0,1,0
34654,gy6686,1.0,0,진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분라이트', ' 진정앰플', ' 부스터앰플', ' 미...",1,0,0,0,1,0,0,0,1,0


In [3]:
df = pd.read_csv('data/CRM_mid.csv', encoding='utf-8-sig', index_col=0)

In [3]:
df.drop_duplicates(['아이디'], keep = 'first', ignore_index=True, inplace=True)

In [4]:
df['쑥앰플'] = None
df['진정앰플'] = None
df['수분앰플'] = None
df['수분라이트'] = None
df['미백앰플'] = None

In [5]:
ampoul = ['쑥앰플', '진정앰플', '수분앰플', '수분라이트', '미백앰플']
ampoul[1]

'진정앰플'

In [6]:
# 수분앰플라이트 -> 수분라이트
df['앰플'] = df['앰플'].str.replace(pat='수분앰플라이트', repl='수분라이트', regex=False)

In [7]:
# 앰플 숫자형태로 인코딩
for i in range(len(df)):
    for j in ampoul:
        if j in df['앰플'][i]:
            df[j][i] = 1
        else:
            df[j][i] = 0


<ipython-input-7-d2d75a472ee5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j][i] = 1
<ipython-input-7-d2d75a472ee5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j][i] = 0


In [8]:
# 앰플 빈칸 행 제거
for i in range(len(df)):

    if (df['진정앰플'][i] == 0 and df['수분앰플'][i] == 0 and df['미백앰플'][i] == 0 and df['수분라이트'][i] == 0 and df['쑥앰플'][i] == 0):
        df.drop(i, 0, inplace=True)

In [11]:
df.to_csv("data/CRM_mid2.csv", encoding='utf-8-sig')

In [12]:
df

,아이디,나이,성별,선호제품,주문상품명,옵션입력,앰플,건성,지성,복합성,...,과잉피지,여드름,좁쌀,홍조&예민,기미&잡티,쑥앰플,진정앰플,수분앰플,수분라이트,미백앰플
0,thsk486,4.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",1,0,0,...,0,0,1,1,0,1,1,0,0,0
1,thsk486,4.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",1,0,0,...,0,0,1,1,0,1,1,0,0,0
2,sjh3460,3.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,1,0,...,0,1,0,0,0,1,1,0,0,0
3,kaeng2da,3.0,0,수분앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,0,1,...,0,0,1,0,0,1,1,0,0,0
4,gmlwlss625,2.0,0,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진정앰플', ' 쑥앰플', ' 진...",0,0,1,...,1,1,1,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34651,nadaseonho,3.0,0,진정앰플,[시앙제이X아이다] 인생 버블토너 1차 공구!,필수 옵션 프로폴리스 진정토너 옵션 머그워트 클리어토너 옵션 ...,[' 진정앰플울트라'],0,0,1,...,0,0,0,1,0,0,1,0,0,0
34652,rarrarra12,3.0,0,진정앰플,BEST앰플3종(수분/미백/진정),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,"[' 수분앰플', ' 미백앰플', ' 진정앰플']",0,1,0,...,1,1,1,0,0,0,1,1,0,1
34653,gy6686,1.0,0,진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분라이트', ' 진정앰플', ' 부스터앰플', ' 미...",1,0,0,...,0,0,0,1,0,0,1,1,1,1
34654,gy6686,1.0,0,진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[' 수분앰플오리지널', ' 수분라이트', ' 진정앰플', ' 부스터앰플', ' 미...",1,0,0,...,0,0,0,1,0,0,1,1,1,1


In [39]:
df1 = df.iloc[:, [1,2,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]]

In [40]:
df1

,나이,성별,건성,지성,복합성,모름,건조함,과잉피지,여드름,좁쌀,홍조&예민,기미&잡티,쑥앰플,진정앰플,수분앰플,수분라이트,미백앰플
0,4.0,0,1,0,0,0,1,0,0,1,1,0,1,1,0,0,0
1,4.0,0,1,0,0,0,1,0,0,1,1,0,1,1,0,0,0
2,3.0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0
3,3.0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0
4,2.0,0,0,0,1,0,0,1,1,1,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34651,3.0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0
34652,3.0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,0,1
34653,1.0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,1,1
34654,1.0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,1,1


In [42]:
x_data = df1.iloc[:, 0:12]
x_data
# 12

,나이,성별,건성,지성,복합성,모름,건조함,과잉피지,여드름,좁쌀,홍조&예민,기미&잡티
0,4.0,0,1,0,0,0,1,0,0,1,1,0
1,4.0,0,1,0,0,0,1,0,0,1,1,0
2,3.0,0,0,1,0,0,0,0,1,0,0,0
3,3.0,0,0,0,1,0,0,0,0,1,0,0
4,2.0,0,0,0,1,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
34651,3.0,0,0,0,1,0,1,0,0,0,1,0
34652,3.0,0,0,1,0,0,0,1,1,1,0,0
34653,1.0,0,1,0,0,0,1,0,0,0,1,0
34654,1.0,0,1,0,0,0,1,0,0,0,1,0


In [45]:
y_data = df1.iloc[:, 12:]
y_data

,쑥앰플,진정앰플,수분앰플,수분라이트,미백앰플
0,1,1,0,0,0
1,1,1,0,0,0
2,1,1,0,0,0
3,1,1,0,0,0
4,1,1,0,0,0
...,...,...,...,...,...
34651,0,1,0,0,0
34652,0,1,1,0,1
34653,0,1,1,1,1
34654,0,1,1,1,1


In [100]:
y_data = pd.get_dummies(y_data)

In [46]:
y_data
# 5

,쑥앰플,진정앰플,수분앰플,수분라이트,미백앰플
0,1,1,0,0,0
1,1,1,0,0,0
2,1,1,0,0,0
3,1,1,0,0,0
4,1,1,0,0,0
...,...,...,...,...,...
34651,0,1,0,0,0
34652,0,1,1,0,1
34653,0,1,1,1,1
34654,0,1,1,1,1


In [49]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=111, stratify=y_data)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
x_train
x_test
y_train
y_test

In [104]:
np.array(x_train)

array([[3., 0., 0., ..., 1., 1., 0.],
       [5., 0., 0., ..., 0., 0., 0.],
       [5., 0., 1., ..., 0., 1., 0.],
       ...,
       [4., 0., 0., ..., 0., 1., 1.],
       [4., 0., 0., ..., 1., 1., 1.],
       [7., 0., 1., ..., 0., 0., 1.]])

In [105]:
X = np.array(x_train)
y = np.array(y_train)

### 아이디 중복제거 x

In [108]:

from tensorflow.keras.models import Sequential # 케라스의 Sequential()을 임포트
from tensorflow.keras.layers import Dense # 케라스의 Dense()를 임포트
from tensorflow.keras import optimizers # 케라스의 옵티마이저를 임포트

model=Sequential()
model.add(Dense(5, input_dim=12, activation='softmax'))
sgd=optimizers.SGD(lr=0.01)

model.compile(optimizer=SGD(lr=0.01) ,loss='categorical_crossentropy',metrics=['accuracy'])
# SGD. RMSprop, Adam
# 손실 함수(Loss function)는 binary_crossentropy(이진 크로스 엔트로피)를 사용합니다.
model.fit(X,y, batch_size=10, epochs=100, shuffle=False, validation_data=(x_test, y_test))
# 주어진 X와 y데이터에 대해서 오차를 최소화하는 작업을 800번 시도합니다.

Epoch 1/100
291/291 [==============================] - 1s 2ms/step - loss: 1.7713 - accuracy: 0.2381 - val_loss: 1.4913 - val_accuracy: 0.3553
Epoch 2/100
291/291 [==============================] - 0s 974us/step - loss: 1.4806 - accuracy: 0.3520 - val_loss: 1.4287 - val_accuracy: 0.4090
Epoch 3/100
291/291 [==============================] - 0s 1ms/step - loss: 1.4295 - accuracy: 0.4012 - val_loss: 1.3943 - val_accuracy: 0.4531
Epoch 4/100
291/291 [==============================] - 0s 993us/step - loss: 1.4008 - accuracy: 0.4206 - val_loss: 1.3737 - val_accuracy: 0.4627
Epoch 5/100
291/291 [==============================] - 0s 945us/step - loss: 1.3831 - accuracy: 0.4323 - val_loss: 1.3605 - val_accuracy: 0.4635
Epoch 6/100
291/291 [==============================] - 0s 920us/step - loss: 1.3716 - accuracy: 0.4386 - val_loss: 1.3516 - val_accuracy: 0.4691
Epoch 7/100
291/291 [==============================] - 0s 955us/step - loss: 1.3638 - accuracy: 0.4456 - val_loss: 1.3454 - val_accura

KeyboardInterrupt: 

In [58]:
print(model.predict(X))
y

[[0.02167521 0.39025295 0.11766967 0.38893887 0.08146333]
 [0.15575294 0.05806572 0.04666207 0.15765631 0.581863  ]
 [0.01128221 0.1144217  0.07389101 0.5320612  0.26834393]
 ...
 [0.02642363 0.00385817 0.00173943 0.12195425 0.8460245 ]
 [0.01386134 0.3088279  0.21166672 0.27622718 0.18941687]
 [0.0090987  0.05246019 0.02071993 0.5577168  0.36000445]]


array([[0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0]], dtype=uint8)